In [31]:
import subprocess
import os
import pandas as pd

In [3]:
CHEXPERT_FOLDER = '~/chexpert/chexpert-labeler'

In [4]:
CHEXPERT_PYTHON = '~/miniconda3/envs/chexpert-label/bin/python'

In [5]:
TMP_FOLDER = '~/medvqa-workspace/tmp'

In [26]:
NEGBIO_PATH = '/home/pamessina/chexpert/NegBio'

In [27]:
def _get_custom_env():
    custom_env = os.environ.copy()
    prev = custom_env.get('PYTHONPATH', '')
    custom_env['PYTHONPATH'] = f'{NEGBIO_PATH}:{prev}'
    return custom_env

In [30]:
input_path = os.path.join(TMP_FOLDER, 'reports-input.csv')
output_path = os.path.join(TMP_FOLDER, 'reports-output.csv')
cmd_cd = f'cd {CHEXPERT_FOLDER}'
cmd_call = f'{CHEXPERT_PYTHON} label.py --reports_path {input_path} --output_path {output_path}'
cmd = f'{cmd_cd} && {cmd_call}'

subprocess.run(
    cmd, shell=True, check=True,
    stdout=subprocess.PIPE, stderr=subprocess.PIPE,
    env=_get_custom_env(),
)

CompletedProcess(args='cd ~/chexpert/chexpert-labeler && ~/miniconda3/envs/chexpert-label/bin/python label.py --reports_path ~/medvqa-workspace/tmp/reports-input.csv --output_path ~/medvqa-workspace/tmp/reports-output.csv', returncode=0, stdout=b'', stderr=b'')

In [14]:
!head /home/pamessina/medvqa-workspace/tmp/reports-input.csv

no free air below the right hemidiaphragm is seen
however , other radiopaque fluid , such as fluid , , , be filling the airspace in this region
there is no free air under the hemidiaphragms . low lung volumes but no acute process and no evidence of free peritoneal air .


In [33]:
len(pd.DataFrame(['asdfasdf', 'asdfasdfasdf', 'asdfasdf']))

3